In [1]:
from PIL import Image
from tqdm import tqdm
import hashlib
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd
import Augmentor

from sklearn.model_selection import StratifiedKFold, KFold

In [2]:
AUGMENT_FACTOR = 3
IMG_SHAPE = (224, 224)

In [3]:
path = Path("dataset_updated")

In [4]:
img_list = []
for img in path.glob('*/*/*'):
    try:
        Image.open(img)
    except:
        continue
    cat = img.parts[-2]
    pathstr = str(img.absolute())
    img_id = hashlib.md5(pathstr.encode('utf-8')).hexdigest()
    img_list.append((img_id, pathstr, cat))
img_df = pd.DataFrame(img_list, columns=['id', 'path', 'category'])
img_df.index = img_df.id
img_df.drop('id', axis=1, inplace=True)
img_df.category = img_df.category.astype('category')
img_df.head()

/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6029312 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1311848 bytes but only got 785. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 8. 
  warnings.warn(str(msg))


,path,category
id,,
96cb78a545ae9bff37048375762a43c4,/home/asm/dev/budokai/dataset_updated/validati...,iconography
c8c4ef86d86573cacb5991e362a4ac2d,/home/asm/dev/budokai/dataset_updated/validati...,iconography
1eac68a564617590992bae5e51fd4d8e,/home/asm/dev/budokai/dataset_updated/validati...,iconography
0ddef4a9979cb84a1855c6e5133bd88c,/home/asm/dev/budokai/dataset_updated/validati...,iconography
08d8de80b60f2373689afa401ad4c547,/home/asm/dev/budokai/dataset_updated/validati...,iconography


In [5]:
def apply_pipeline(df, sample_size):
    
    p = Augmentor.DataFramePipeline(df, 'path', 'category')
    p.resize(1, *IMG_SHAPE)
    
    samples = p.augmentor_images
    if sample_size > len(samples):
        extra = np.random.choice(p.augmentor_images, sample_size - len(samples))
        samples = np.concatenate((samples, extra), axis=0)
        
    with tqdm(total=len(samples), desc="Generating samples", unit="samples") as pbar:
        
        def process(img):
            aug_img = np.array(p._execute(img, save_to_disk=False))
            if aug_img.ndim < 3:
                aug_img = np.stack([aug_img]*3, -1)
            if aug_img.shape[-1] > 3:
                aug_img = aug_img[..., :3]
            assert aug_img.shape == (*IMG_SHAPE, 3), img.image_path
            pbar.update(1)
            return aug_img, img.categorical_label
        
        X, y = zip(*map(process, samples))
    
    return np.array(X), np.array(y)

In [6]:
skf = StratifiedKFold(5)
for i, (train, test) in enumerate(skf.split(img_df.path, img_df.category)):
    X_train, y_train = apply_pipeline(img_df.iloc[train], AUGMENT_FACTOR * len(train))
    X_test, y_test = apply_pipeline(img_df.iloc[test], AUGMENT_FACTOR * len(test))
    np.savez(f'fold{i}_data.npz', X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
    print()

Generating samples:   0%|          | 25/20580 [00:00<01:23, 246.60samples/s]

Initialised with 6860 image(s) found.
Output directory set to output.

Generating samples:   1%|          | 44/5151 [00:00<00:11, 427.76samples/s]

Initialised with 1717 image(s) found.
Output directory set to output.

Generating samples: 100%|██████████| 5151/5151 [00:12<00:00, 416.29samples/s]


...

Generating samples:   0%|          | 44/20592 [00:00<00:48, 424.93samples/s]

Initialised with 6864 image(s) found.
Output directory set to output.

Generating samples:   1%|          | 34/5139 [00:00<00:15, 323.40samples/s]

Initialised with 1713 image(s) found.
Output directory set to output.

Generating samples: 100%|██████████| 5139/5139 [00:12<00:00, 415.91samples/s]
